In [60]:
''' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'''


' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [3]:
categories = ['adoptable', 'adopted']

In [4]:
path_csv = '../data/csv/fullDB_valid_csv.csv'
df1 = pd.read_csv(path_csv)

#impute empty records
df = df1.fillna("None")

#not necessary cleaning
df.drop(['photos', 'videos', 'distance', 'status_changed_at', 'published_at', 'distance', 'contact', 'organization_animal_id', 'type', 'photos'], axis = 1, inplace= True)

df = pd.get_dummies(df, columns=['status'])
df.drop("status_adoptable", axis = 1, inplace=True)
# print(df.shape)

df_content = df[["status_adopted", "description"]].copy()
# print(df_content.head())

X = df_content["description"] #data
X.to_numpy()
# print(X)

y = df_content["status_adopted"] #target
y.to_numpy()
# print(y)  
# 0 == negative,  1 == positive                                     

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,

In [5]:
print(X[0])

Meet Ruff, Ruff is a smaller breed - probably around 10 lbs at  6-7  months old and a complete mix...


In [6]:
# print(df_content.head())
# print(X)
# print(y)
print(len(X)) #722
print(len(y)) #722

722
722


In [32]:
import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
new_stopwords = ['old', 'mix', 'dogs', '039', 'amp', 'sweet', 'year', 'years', 'website' , 'loves', 'adoption', 'application', 'shelter', 'rescue', 'rescued']
new_stopwords_list = stop_words.union(new_stopwords)

count_vect = CountVectorizer(lowercase=True, tokenizer=None, stop_words = new_stopwords_list,  analyzer='word', max_df=1.0, min_df=1,  max_features=None) 
                             
count_vect.fit(X)

target_names = y

In [31]:
print(count_vect)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words={'039', 'a', 'about', 'above', 'adoption', 'after',
                            'again', 'against', 'ain', 'all', 'am', 'amp', 'an',
                            'and', 'any', 'application', 'are', 'aren',
                            "aren't", 'as', 'at', 'be', 'because', 'been',
                            'before', 'being', 'below', 'between', 'both',
                            'but', ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [34]:

X_counts = count_vect.transform(X)
print("The type of X_counts is {0}.".format(type(X_counts)))
print("The X matrix has {0} rows (documents) and {1} columns (words).".format(
        X_counts.shape[0], X_counts.shape[1]))

The type of X_counts is <class 'scipy.sparse.csr.csr_matrix'>.
The X matrix has 722 rows (documents) and 1806 columns (words).


In [10]:
tfidf_transformer = TfidfTransformer(use_idf=True)
tfidf_transformer.fit(X_counts)
X_tfidf = tfidf_transformer.transform(X_counts)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y.values, test_size=0.2, random_state=42)

In [12]:
print(X_train.shape)
print(y_train.shape)

(577, 1806)
(577,)


In [13]:
print(X_test.shape)
print(y_test.shape)

(145, 1806)
(145,)


In [14]:
nb_model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nb_model.fit(X_train, y_train);


In [15]:
feature_words = count_vect.get_feature_names()
n = 20 #number of top words associated with the category that we wish to see

for cat in range(len(categories)): #categories == ['adoptable', 'adopted']
    print(f"\nTarget: {cat}, name: {target_names[cat]}")
    log_prob = nb_model.feature_log_prob_[cat]
    i_topn = np.argsort(log_prob)[::-1][:n]
    features_topn = [feature_words[i] for i in i_topn]
    print(f"Top {n} tokens: ", features_topn)



Target: 0, name: 1
Top 20 tokens:  ['none', 'good', 'dog', 'girl', 'meet', 'female', 'pit', 'foster', 'name', 'pounds', 'please', 'friendly', 'cats', 'boy', 'around', 'weight', 'fill', 'breed', 'spayed', 'playful']

Target: 1, name: 1
Top 20 tokens:  ['online', 'none', 'fill', 'official', 'meet', 'home', 'looking', 'dog', 'forever', 'puppy', 'name', 'lbs', 'male', 'please', 'hi', '20', 'came', 'months', 'shepherd', 'month']


In [42]:
feature_words = count_vect.get_feature_names()
# print(feature_words)
n = 10 #number of top words associated with the category that we wish to see
feat_val_dict = {}

for cat in range(len(categories)): #categories == ['adoptable', 'adopted']
    # print(cat)
    print(f"\nTarget: {cat}, name: {target_names[cat]}")
    log_prob = nb_model.feature_log_prob_[cat]
    # print(nb_model.feature_log_prob_[cat])
    i_topn = np.argsort(log_prob)[::-1][:n]
    features_topn = [feature_words[i] for i in i_topn]
    print(f"Top {n} tokens: ", features_topn, i_topn)
    # for word in features_topn:
    #     print(word)



Target: 0, name: 1
Top 10 tokens:  ['none', 'good', 'dog', 'girl', 'meet', 'female', 'pit', 'foster', 'name', 'pounds'] [1132  730  548  720 1038  649 1242  670 1099 1273]

Target: 1, name: 1
Top 10 tokens:  ['online', 'none', 'fill', 'official', 'meet', 'home', 'looking', 'dog', 'forever', 'puppy'] [1162 1132  653 1148 1038  793  976  548  666 1307]


In [75]:
y_predicted = nb_model.predict(X_test)
# X_test[0]

In [76]:
# y_predicted

In [77]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predicted)


0.8547008547008548

In [78]:
y_pred_proba = nb_model.predict_proba(X_test)[:,1]
y_pred_proba

array([0.71216066, 0.75930515, 0.86844199, 0.90706522, 0.82932451,
       0.87140013, 0.67993514, 0.97125573, 0.57241605, 0.7893452 ,
       0.72058565, 0.8193681 , 0.64633598, 0.64088113, 0.78631544,
       0.59910174, 0.72793925, 0.42637203, 0.74771564, 0.97826375,
       0.87621175, 0.83847934, 0.42637203, 0.97125573, 0.82459837,
       0.5622289 , 0.75365443, 0.88270828, 0.74722437, 0.78644864,
       0.66062795, 0.91917311, 0.92825488, 0.42637203, 0.71207837,
       0.63259685, 0.42637203, 0.64736833, 0.81956706, 0.79830218,
       0.81997776, 0.8271789 , 0.84080842, 0.97395362, 0.42637203,
       0.77029561, 0.97125573, 0.83654551, 0.60073157, 0.72273535,
       0.6704058 , 0.67639054, 0.69067067, 0.7889291 , 0.90553464,
       0.90292565, 0.29503247, 0.69614049, 0.83462137, 0.78973342,
       0.72599417, 0.63641401, 0.72587675, 0.83836189, 0.67036371,
       0.70725115, 0.77720308, 0.64089543, 0.85864731, 0.63837743,
       0.61157425, 0.90212459, 0.88829324, 0.6133927 , 0.65052

In [84]:
thresh=0.7
y_pred = (y_pred_proba>=thresh).astype(int)
y_pred

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1])

0.7783251231527093

In [80]:
# next steps:
# 1. pickle and save the model 
import pickle
save_documents = open("pickled_algos/pickled_nb.pickle","wb")
pickle.dump(nb_model, save_documents)
save_documents.close()

tfidf = tfidf_transformer
save_documents = open("pickled_algos/tfidf_transformer.pickle", "wb")
pickle.dump(tfidf, save_documents)
save_documents.close()

count_vect = count_vect
save_documents = open("pickled_algos/count_vect.pickle", "wb")
pickle.dump(count_vect, save_documents)
save_documents.close()



In [88]:
# 2. test the model by 
#turn this whoel cell into a function that takes in a string
#predict_one
with open('pickled_algos/pickled_nb.pickle', 'rb') as f:
    model = pickle.load(f)

with open('pickled_algos/tfidf_transformer.pickle', 'rb') as f:
    tfidf = pickle.load(f)

with open('pickled_algos/count_vect.pickle', 'rb') as f:
    cv = pickle.load(f)

# string_pred = ['this girl is a foster pit and has none of her teeth']  
string_pred = ['fill out an online form to find this male puppy a forever home']

cv_transformed = cv.transform(string_pred) #counts how many words
tfidf_transformed = tfidf.transform(cv_transformed)  #tf == cv . 
string_predicted = model.predict(tfidf_transformed) 
res = str(string_predicted[0])
if res == '0':
    print('Negative Sentiment')
else:
    print("Positive Sentiment")

Positive Sentiment


In [89]:
def predict_one(input):
    with open('pickled_algos/pickled_nb.pickle', 'rb') as f:
        model = pickle.load(f)

    with open('pickled_algos/tfidf_transformer.pickle', 'rb') as f:
        tfidf = pickle.load(f)

    with open('pickled_algos/count_vect.pickle', 'rb') as f:
        cv = pickle.load(f)

  
    cv_transformed = cv.transform(string_pred) #counts how many words
    tfidf_transformed = tfidf.transform(cv_transformed)  #tf == cv . 
    string_predicted = model.predict(tfidf_transformed) 
    res = str(string_predicted[0])
    if res == '0':
        res = ('Negative Sentiment')
    else:
        res = ("Positive Sentiment")
    return res

# string_pred = ['this girl is a foster pit and has none of her teeth']
string_pred = ['fill out an online form to find this male puppy a forever home']

predict_one(string_pred)

'Positive Sentiment'